In [1]:
import pandas as pd

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA  # for compactness

In [3]:
pd.set_option('max.columns', 999)

In [4]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alexanderdesouza/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
raw_headlines = pd.read_csv('../data/abcnews_million_headlines.csv')

In [6]:
raw_headlines = raw_headlines.sample(1000)['headline'].values
raw_headlines[:10]

array(['woman loses fingertips in car accident',
       'peaches geldof autops inconclusive',
       'taxpayers to pay 1b for desal plant', 'diary the amazon of asia',
       'aussie cricketers disciplined in england',
       'wwf report critical of queensland land clearing return',
       'guilty plea over two illegal investment schemes',
       "pontville's future uncertain", 'afl charges 19 over melee',
       'gungahlin development to improve services'], dtype=object)

In [ ]:
tagged_data = [TaggedDocument(words=nltk.word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(raw_headlines)]
tagged_data[:5]

In [ ]:
# Here is a list of unique tags
# ...tagged_data[:1][0].tags[0]...

In [ ]:
# Define a set of hyperparameters that can be optimized later
max_epochs = 100  # number of training epochs
alpha = 0.025     # initial learning rate, selected 

In [ ]:
Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)

model = Doc2Vec(size=10,            # let's call it something like the number of neurons
                alpha=alpha,        # learning rate
                min_alpha=0.00025,  # minimum learning rate
                min_count=1,        # minimum term frequency
                dm =0.5)            # there is a trade off here in the degree of memory distribution to use for the model (i.e., DM v DBOW)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # Decrease the learning rate
    model.alpha -= 0.0002
    # Fix the learning rate to prevent decay
    model.min_alpha = model.alpha

In [ ]:
# Recall this document in the corpus, the first entry in the random sampling performed above...
# ['boy', 'to', 'face', 'court', 'accused', 'of', 'knife', 'threats']
# A query might include a variation we construct as shown below

The examples here probably won't match directly anymore after loading and running this notebook from scratch.

In [ ]:
tokenized_query_statement = "young man accused of violent threats to appear before judge".split()
query_vector = model.infer_vector(tokenized_query_statement)
similarity = model.docvecs.most_similar([query_vector])
similarity

In [ ]:
matched_headline_indexes = [int(i) for i, s in similarity]
raw_headlines[matched_headline_indexes]

In [ ]:
np.argmax([-0.1, -0.2, -0.3])